In [1]:
import pandas as pd

df = pd.read_csv('../../data/train.csv')

print(len(df))

df.head(3)

26371


,conversation_id,message,sentiment
0,5752,do you like dogs?,Curious to dive deeper
1,8173,WOw good for him. Did you know in Guatemala re...,Curious to dive deeper
2,5152,Yes he became a professional driver by beating...,Curious to dive deeper


In [3]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)

dataset

/Users/user/anaconda3/envs/splore/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['conversation_id', 'message', 'sentiment', 'label'],
    num_rows: 26371
})

In [ ]:
df = pd.read_csv('../../data/train.csv')

In [12]:
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset, DatasetDict
import pandas as pd

label_mapping = {
    'Curious to dive deeper': 0,
    'Neutral': 1,
    'Surprised': 2,
    'Happy': 3,
    'Disgusted': 4,
    'Sad': 5,
    'Fearful': 6,
    'Angry': 7
}

df = pd.read_csv('../../data/train.csv')
df['label'] = df['sentiment'].map(label_mapping)

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split the dataset into training and validation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
dataset_dict = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

# Function to remove unnecessary columns and set format
def format_dataset(dataset):
    # Check for the unnecessary column and remove it if exists
    columns = dataset.column_names
    if '__index_level_0__' in columns:
        dataset = dataset.remove_columns(['__index_level_0__'])
    # Rename 'label' column to 'labels'
    dataset = dataset.rename_column('label', 'labels')
    # Set the dataset format to torch with required columns
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return dataset

# Apply the format_dataset function to both train and test sets
dataset_dict['train'] = format_dataset(dataset_dict['train'])
dataset_dict['test'] = format_dataset(dataset_dict['test'])

# Load the model and modify it for 8 labels
model = AutoModelForSequenceClassification.from_pretrained(
    "SamLowe/roberta-base-go_emotions",
    num_labels=8,
    ignore_mismatched_sizes=True
)

device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reduce batch size to reduce memory usage
    per_device_eval_batch_size=8,  # Reduce batch size to reduce memory usage
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")

















Map: 100%|██████████| 26371/26371 [00:02<00:00, 11452.59 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at SamLowe/roberta-base-go_emotions and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: MPS backend out of memory (MPS allocated: 18.14 GB, other allocations: 704.00 KB, max allowed: 18.13 GB). Tried to allocate 12.00 KB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).